# Auto-GPT Style Agent with Explicit CoT (Single-File, No web_tools Install)

This notebook shows how you can implement an **Auto-GPT** style agent with explicit **Chain-of-Thought** in a **single file**, **no** external package like `web_tools` needed:

1. **Inline Web Tools**: We define a dummy `web_search_tool` and `web_scrape_tool` as normal Python functions.
2. **OpenAI CoT Adapter**: We use the `openai` library to call your LLM model.
3. **Auto-GPT CoT Agent**: Repeatedly calls the LLM to see if a `FINAL_ANSWER` is ready or if we need to **PLAN_TO_VISIT** certain sites.
4. **Main**: Runs the scenario with a user query.

We **mock** two sites:
- `review_evs.com` – Contains user sentiment data.
- `auto_market.com` – Contains approximate pricing.


## 1. Inline “Web Tools”
We define two Python functions:
- `web_search_tool(query)`: returns a **list** of relevant site URLs
- `web_scrape_tool(url)`: returns **text content** from that site

No extra installation needed. We just store data in a dictionary.

In [ ]:
MOCK_WEBSITES = {
    "review_evs.com": (
        "Buyer Sentiment Summary:\n"
        " - Tesla Model Y: Highly rated by most users, praising range and performance.\n"
        " - Nissan Leaf: Moderate reviews, good for city driving but limited range.\n"
        " - E-Car Pro: Newer entrant, positive early feedback on interior.\n"
        "Overall, the top-rated EV by user sentiment is the Tesla Model Y."
    ),
    "auto_market.com": (
        "Pricing Data:\n"
        " - Tesla Model Y: Starting at $54,000.\n"
        " - Nissan Leaf: Starting at $28,000.\n"
        " - E-Car Pro: Starting at $39,000.\n"
    )
}

def web_search_tool(query):
    """
    Returns a list of site URLs based on the keywords in `query`.
    """
    results = []
    ql = query.lower()
    if "review" in ql or "sentiment" in ql:
        results.append("review_evs.com")
    if "price" in ql or "market" in ql:
        results.append("auto_market.com")
    # fallback
    if not results:
        results.append("review_evs.com")
    return results

def web_scrape_tool(url):
    """
    Returns mock text content for a site.
    """
    return MOCK_WEBSITES.get(url, "No content for this URL.")

## 2. OpenAI CoT Adapter
We define a function that calls **OpenAI** for chain-of-thought. It concatenates the conversation log into a single user message, then returns the LLM’s answer.

**Note**: Make sure you have `openai` installed (`pip install openai`) and your environment variable `OPENAI_API_KEY` set, or pass an API key in code.

In [ ]:
from openai import OpenAI
import os
import re

os.environ['OPENAI_API_KEY']=''


def call_openai_for_cot(conversation, model="gpt-4o", temperature=0.2):
    """
    Calls OpenAI to get chain-of-thought or final answer.
    The conversation is a list of strings.
    """
    messages = [
        {"role": "system", "content": "You are an Auto-GPT style agent with explicit chain-of-thought."},
        {"role": "user", "content": "\n".join(conversation)}
    ]

    client = OpenAI()

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=300,
    )

    return response.choices[0].message.content.strip()

## 3. Auto-GPT CoT Agent
This agent:
1. Maintains `conversation_log`.
2. Each iteration, calls `call_openai_for_cot` with the entire log.
3. If it finds `FINAL_ANSWER:`, it returns.
4. If it finds `PLAN_TO_VISIT: [ ... ]`, it extracts keywords, calls `web_search_tool`, then `web_scrape_tool`, and appends results.
5. Continues until `max_iterations` is reached or we get a final answer.

In [ ]:
class AutoGPTCoTAgent:
    def __init__(self, max_iterations=5, model="gpt-4o"):
        self.conversation_log = []
        self.visited_sites = set()
        self.max_iterations = max_iterations
        self.model = model

    def run(self, user_query):
        # Add user query to the conversation
        self.conversation_log.append(f"User: {user_query}")

        for _ in range(self.max_iterations):
            # 1) Call OpenAI for chain-of-thought
            llm_response = call_openai_for_cot(self.conversation_log, model=self.model)
            self.conversation_log.append(f"Agent: {llm_response}")

            # 2) Check if we have FINAL_ANSWER
            if "FINAL_ANSWER:" in llm_response:
                answer_text = llm_response.split("FINAL_ANSWER:")[-1].strip()
                return answer_text

            # 3) Check for PLAN_TO_VISIT:
            match = re.search(r"PLAN_TO_VISIT:\s*\[(.*?)\]", llm_response)
            if match:
                plan_str = match.group(1)
                # e.g. "review sentiment" or "price market"
                keywords = [k.strip() for k in plan_str.split(",")]

                for kw in keywords:
                    # web search
                    sites = web_search_tool(kw)
                    self.conversation_log.append(f"System: Searching for '{kw}' -> {sites}")

                    for site in sites:
                        if site not in self.visited_sites:
                            content = web_scrape_tool(site)
                            self.visited_sites.add(site)
                            self.conversation_log.append(f"System: SCRAPED {site} -> {content}")
            else:
                # no plan found, we just continue
                self.conversation_log.append("System: No plan found. Continuing.")

        return "Sorry, couldn't complete the research in time."

## 4. Main Orchestration
Finally, we provide the user query: "Find the price of the best EV in the market based on buyer sentiment." The agent attempts to gather enough info from the dummy websites, then produce a final answer.

In [ ]:
if __name__ == "__main__":
    user_query = "Find the price of the best EV in the market based on buyer sentiment."
    agent = AutoGPTCoTAgent(max_iterations=5, model="gpt-4o")
    final_answer = agent.run(user_query)

    print("=== FINAL ANSWER ===")
    print(final_answer)
    
    # Uncomment to see the conversation log:
    for line in agent.conversation_log:
        print(line)

## How to Run
1. **Save** this file as `auto_gpt_cot_demo_singlefile.ipynb`.
2. **Install** `openai` via `pip install openai`.
3. **Set** `OPENAI_API_KEY` in your environment or do `openai.api_key = "<your_key>"` in code.
4. **Run all cells**. The last cell attempts to iterate until it finds a final answer or hits `max_iterations`.

## Notes
1. **No** separate `web_tools` install—just local Python functions.
2. **Chain-of-Thought** is an **explicit** style: we prompt the LLM to produce lines like `CHAIN_OF_THOUGHT:` and `PLAN_TO_VISIT:` or `FINAL_ANSWER:`.
3. **Dummy** data for websites. In production, you'd fetch real data from the web or a knowledge store.
4. You may need to refine the prompt for your actual LLM usage, especially if it doesn't consistently follow the `PLAN_TO_VISIT` or `FINAL_ANSWER` pattern.
